In [ ]:
!pip install gensim --upgrade
!pip install --upgrade pip
!pip install --upgrade jupyter notebook
!pip install pyLDAvis

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
# DataFrame
import pyLDAvis
import pandas as pd
import tensorflow as tf
# Matplot
import matplotlib.pyplot as plt
%matplotlib inline

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

# Keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from tensorflow.keras import utils
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import pyLDAvis.gensim_models as gensimvis
!pip install gensim --upgrade
!pip install --upgrade pip
!pip install --upgrade jupyter notebook
!pip install pyLDAvis
# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

# Word2vec
import gensim

# Utility
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools


/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecate

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('stopwords')
# TEXT CLENAING
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

# WORD2VEC 
W2V_SIZE = 64
W2V_WINDOW = 10
W2V_EPOCH = 256
W2V_MIN_COUNT = 10

# KERAS
SEQUENCE_LENGTH = 300
EPOCHS = 10
BATCH_SIZE = 64


# EXPORT
KERAS_MODEL = "model.h5"
WORD2VEC_MODEL = "model.pkl"
TOKENIZER_MODEL = "tokenizer.pkl"
ENCODER_MODEL = "encoder.pkl"

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<input>:3: DeprecationWarning: invalid escape sequence \S
<input>:3: DeprecationWarning: invalid escape sequence \S
<input>:3: DeprecationWarning: invalid escape sequence \S
<input>:3: DeprecationWarning: invalid escape sequence \S
<input>:3: DeprecationWarning: invalid escape sequence \S
<input>:3: DeprecationWarning: invalid escape sequence \S
<input>:3: DeprecationWarning: invalid escape sequence \S
<input>:3: DeprecationWarning: invalid escape sequence \S
<input>:3: DeprecationWarning: invalid escape sequence \S
<input>:3: DeprecationWarning: invalid escape sequence \S
<input>:3: DeprecationWarning: invalid escape sequence \S
<input>:3: DeprecationWarning: invalid escape sequence \S
<input>:3: DeprecationWarning: invalid escape sequence \S
<input>:3: DeprecationWarning: invalid escape sequence \S
<input>:3: DeprecationWarning: invalid escape sequence \S
<input>:3: DeprecationWarning: invalid escape sequence \S
<input>:3: DeprecationWarning: invalid escape sequence \S
<input>:3: Dep

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset/sentiment_analysis_dataset/Electronics_5.csv')

In [ ]:
df.head(3)

,Unnamed: 0,overall,reviewText,target
0,0,5,We got this GPS for my husband who is an (OTR)...,pos
1,1,1,"I'm a professional OTR truck driver, and I bou...",neg
2,2,3,"Well, what can I say. I've had this unit in m...",natural


In [ ]:
df = df.drop(['Unnamed: 0' , 'overall'] , axis = 1)

In [ ]:
df1 = df[df['target'] == 'pos'][:40000]
df2 = df[df['target'] == 'natural'][:50000]
df3 = df[df['target'] == 'neg'][:40000]


In [ ]:
df = df1.append([df2 ,df3 ] , sort = True )

In [ ]:

## download stopwords 
stop_words = stopwords.words("english")
## download steaming 
stemmer = SnowballStemmer("english")

## reg for remove special character 
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"


"""
  preprocess function 

  input  : each row from data 

  output : text cleaning 

"""



def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words and len(token) > 3:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)


<input>:8: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \S
<input>:8: DeprecationWarning: invalid escape sequence \S
<ipython-input-10-dc2440d51e03>:8: DeprecationWarning: invalid escape sequence \S
  TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"


In [ ]:
 df.reviewText  = df.reviewText.apply(lambda x: preprocess(x))


In [ ]:
df.reviewText[1]

'professional truck driver bought truck stop hoping make life easier rand mcnally listening first thing charging connect laptop install software attempt update software detected problem update wanted home address could sent patch card hello think unusual home address friend checks weekly might check every months live truck truck stops need make patch available card send cards truck stops devices sold update program multiple times program said completely updated programmed height length weight told preferred highways parked truck stop cincinnati area next pickup miles freeway side couple blocks cell phone sprint said freeway pickup routed thru miles residential streets finally getting pickup exciting especially since every time refused turn street posted trucks took almost minutes figure route happened multiple times short trip decided give another chance pickup north side cincinnati needed head phoenix easy route drive west south intersection indeed cell phone advised however wanted ro

In [ ]:
pattern = r'\b[^\d\W]+\b'
tokenizer = RegexpTokenizer(pattern)
en_stop = stop_words
lemmatizer = WordNetLemmatizer()

In [ ]:
# list for tokenized documents in loop
texts = []

# loop through document list
for i in df['reviewText'].iteritems():
    # clean and tokenize document string
    raw = str(i[1]).lower()
    tokens = tokenizer.tokenize(raw)
    # add tokens to list
    texts.append(tokens)



#texts = [text for text in texts if len(text) > 2]




In [ ]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

## <a> Save Dictionary And Corbus Model </a>

In [ ]:
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

## <a> Upload Dictionary and Corpus  on curl </a> 

In [ ]:

!wget https://transfer.sh/1XLvgMv/corpus.pkl

https://transfer.sh/1XLvgMv/corpus.pkl

In [ ]:

!wget https://transfer.sh/1FJt6lA/dictionary.gensim

https://transfer.sh/1FJt6lA/dictionary.gensim

In [ ]:
print(len(corpus))

130000


## <a> Train LDA model  </a>

In [ ]:
#import pprint
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)

lda_viz = gensimvis.prepare(ldamodel, corpus, dictionary)

lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.124768  0.054488       1        1  41.452189
1     -0.003123 -0.113284       2        1  32.956264
2     -0.121645  0.058796       3        1  25.591547, topic_info=            Term          Freq         Total Category  logprob  loglift
1401      camera  46717.000000  46717.000000  Default  30.0000  30.0000
1329       sound  33090.000000  33090.000000  Default  29.0000  29.0000
4628        lens  21344.000000  21344.000000  Default  28.0000  28.0000
4373       canon  11945.000000  11945.000000  Default  27.0000  27.0000
2758  headphones  13834.000000  13834.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
332         also   7375.460778  25721.689720   Topic3  -5.4508   0.1137
149       bought   6589.571004  30734.095445   Topic3  -5.5635  -0.1770
145         used   6001.564757  20671.1

## <a> Save LDA model </a>

In [ ]:
ldamodel.save('lda.gensim')

## <a> Upload LDA model  on curl </a> 

In [ ]:
!wget https://transfer.sh/1GxKt3Y/lda.gensim

https://transfer.sh/1GxKt3Y/lda.gensim

## <a>  Visualize Topic Modeling </a>

In [ ]:
pyLDAvis.enable_notebook()
#pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
lda_viz = gensimvis.prepare(ldamodel, corpus, dictionary)
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.124768  0.054488       1        1  41.452307
1     -0.003123 -0.113284       2        1  32.956404
2     -0.121645  0.058796       3        1  25.591289, topic_info=            Term          Freq         Total Category  logprob  loglift
1401      camera  46716.000000  46716.000000  Default  30.0000  30.0000
1329       sound  33090.000000  33090.000000  Default  29.0000  29.0000
4628        lens  21344.000000  21344.000000  Default  28.0000  28.0000
4373       canon  11945.000000  11945.000000  Default  27.0000  27.0000
2758  headphones  13834.000000  13834.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
332         also   7375.386281  25721.679304   Topic3  -5.4508   0.1137
149       bought   6589.504445  30734.113749   Topic3  -5.5635  -0.1770
145         used   6001.504137  20671.146471   Topic3  -5.6569   0.1262
152        price   5833.213234  19479.091067   Topic3  -5.6854   0.1571
43          time   6157.592451  32630.109887   Topic3  -5.6313  -0.3046

[205 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
332       1  0.387883    also
332       2  0.325368    also
332       3  0.286723    also
775       1  0.557148  amazon
775       2  0.274402  amazon
...     ...       ...     ...
58        3  0.139581   works
177       1  0.429855   would
177       2  0.340371   would
177       3  0.229767   would
2722      3  0.999936    zoom

[318 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

## <a> load corpus and Dictionary model </a> 

In [ ]:
dictionary1 = gensim.corpora.Dictionary.load('/content/dictionary.gensim')
corpus1 = pickle.load(open('/content/corpus.pkl', 'rb'))
lda10 = gensim.models.ldamodel.LdaModel.load('/content/lda.gensim')


pyLDAvis.enable_notebook()
#pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
lda_viz = gensimvis.prepare(lda10, corpus1, dictionary1)
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.124768  0.054488       1        1  41.452418
1     -0.003123 -0.113284       2        1  32.956382
2     -0.121645  0.058796       3        1  25.591200, topic_info=            Term          Freq         Total Category  logprob  loglift
1401      camera  46716.000000  46716.000000  Default  30.0000  30.0000
1329       sound  33090.000000  33090.000000  Default  29.0000  29.0000
4628        lens  21344.000000  21344.000000  Default  28.0000  28.0000
4373       canon  11945.000000  11945.000000  Default  27.0000  27.0000
2758  headphones  13834.000000  13834.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
332         also   7375.360612  25721.674774   Topic3  -5.4508   0.1137
149       bought   6589.481511  30734.117392   Topic3  -5.5635  -0.1770
145         used   6001.483249  20671.143654   Topic3  -5.6569   0.1262
152        price   5833.192933  19479.073365   Topic3  -5.6854   0.1571
43          time   6157.571021  32630.134764   Topic3  -5.6313  -0.3046

[205 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
332       1  0.387883    also
332       2  0.325368    also
332       3  0.286723    also
775       1  0.557147  amazon
775       2  0.274402  amazon
...     ...       ...     ...
58        3  0.139581   works
177       1  0.429855   would
177       2  0.340371   would
177       3  0.229749   would
2722      3  0.999940    zoom

[318 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])